<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_qa_albert_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup
To get the source code of transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [1]:
!uname -a
!apt-get install file

Linux 1020d5c949f9 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.4 [22.1 kB]
Fetched 275 kB in 1s (475 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading database ... 144467 files and directories currently installed.)
Prepa

In [2]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

     |████████████████████████████████| 675kB 34.7MB/s 
     |████████████████████████████████| 1.1MB 46.1MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
     |████████████████████████████████| 3.8MB 45.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f90cc14e966ba6650154218462fef80068a34d5502ffd9422512480022a579d7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


In [4]:
!ls -al
!rm -rf nlp-qa
!git clone --depth 1 https://github.com/hailusong/nlp-qa.git

total 16
drwxr-xr-x 1 root root 4096 May 29 18:19 .
drwxr-xr-x 1 root root 4096 Jun  3 22:50 ..
drwxr-xr-x 1 root root 4096 Jun  2 16:14 .config
drwxr-xr-x 1 root root 4096 May 29 18:19 sample_data
Cloning into 'nlp-qa'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [5]:
!pip install -r nlp-qa/question-answering/requirements.txt

     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 204kB 11.7MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=4acc7d7f8b42de382af220cc3b7165cc2f5b03fd4de45329423e6d0a6aa39eeb
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
ERROR: pytorch-lightning 0.7.3 has requirement future>=0.17.1, but you'll have future 0.16.0 which is incompatible.


## Download
### Pre-trained ALBERT v2 model
https://huggingface.co/elgeish/cs224n-squad2.0-albert-xxlarge-v1

In [0]:
from transformers import ( AutoTokenizer, AutoModelForQuestionAnswering )

tokenizer = AutoTokenizer.from_pretrained("elgeish/cs224n-squad2.0-albert-xxlarge-v1")
model = AutoModelForQuestionAnswering.from_pretrained("elgeish/cs224n-squad2.0-albert-xxlarge-v1")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


## Inference
Source: https://pytorch.org/hub/huggingface_pytorch-transformers/


In [11]:
# model = torch.hub.load('huggingface/pytorch-transformers', 'modelForQuestionAnswering', 'bert-large-uncased-whole-word-masking-finetuned-squad')
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-large-uncased-whole-word-masking-finetuned-squad')

# The format is paragraph first and then question
text_1 = "Jim Henson was a puppeteer"
text_2 = "Who was Jim Henson ?"
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])

# Predict the start and end positions logits
with torch.no_grad():
    start_logits, end_logits = model(tokens_tensor, token_type_ids=segments_tensors)

# get the highest prediction
answer = tokenizer.decode(indexed_tokens[torch.argmax(start_logits):torch.argmax(end_logits)+1])
assert answer == "puppeteer"

# Or get the total loss which is the sum of the CrossEntropy loss for the start and end token positions (set model to train mode before if used for training)
start_positions, end_positions = torch.tensor([12]), torch.tensor([14])
multiple_choice_loss = model(tokens_tensor, token_type_ids=segments_tensors, start_positions=start_positions, end_positions=end_positions)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


In [19]:
torch.argmax(start_logits), torch.argmax(end_logits)

(tensor(5), tensor(6))

In [26]:
len(indexed_tokens), tokens_tensor.shape

(14, torch.Size([1, 14]))